In [1]:
import pandas as pd
df = pd.read_csv('hmnist_64_64_RGB.csv')

In [2]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel12279,pixel12280,pixel12281,pixel12282,pixel12283,pixel12284,pixel12285,pixel12286,pixel12287,label
0,191,152,194,191,153,195,192,149,192,194,...,185,154,180,186,156,184,182,152,173,2
1,24,13,23,24,14,28,37,24,46,61,...,26,15,29,27,15,28,24,13,25,2
2,185,129,140,192,136,151,198,142,156,198,...,152,120,138,144,113,123,115,81,84,2
3,24,11,19,36,19,30,64,38,50,91,...,27,13,16,25,12,16,24,9,14,2
4,138,94,117,158,113,138,178,133,161,191,...,140,106,118,108,77,92,67,40,55,2


In [3]:
df.shape

(10015, 12289)

In [4]:
x = df.drop("label", axis=1).values
y = df["label"].values

In [5]:
x.shape, y.shape

((10015, 12288), (10015,))

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [7]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8012, 12288), (2003, 12288), (8012,), (2003,))

In [8]:
x_train = x_train.reshape(x_train.shape[0], *(64, 64, 3))
x_test = x_test.reshape(x_test.shape[0], *(64, 64, 3))

In [9]:
x_train.shape, x_test.shape

((8012, 64, 64, 3), (2003, 64, 64, 3))

In [10]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [11]:
y_train.shape, y_test.shape

((8012, 7), (2003, 7))

In [12]:
from keras.applications.mobilenet import MobileNet

In [13]:
base_model = MobileNet(weights="imagenet", include_top=False , input_shape=(64, 64, 3))

In [14]:
from keras.models import Sequential , Model
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

x = base_model.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(7, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 32, 32, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 32, 32, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 32, 32, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 32, 32, 32)       128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 4, 4, 512)        2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 4, 4, 512)         0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 4, 4, 512)        4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 4, 4, 512)        2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 4, 4, 512)         0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 4, 4, 512)         262144    
                                                                 
 conv_pw_9

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_accuracy',patience=5,mode='max',restore_best_weights=True)

In [18]:
hist = model.fit(x_train,y_train,epochs=5,batch_size=32,verbose=1,validation_split=0.2,callbacks=es)

Epoch 1/5
201/201 [==============================] - 114s 554ms/step - loss: 1.1627 - accuracy: 0.6477 - val_loss: 1.3204 - val_accuracy: 0.7118
Epoch 2/5
201/201 [==============================] - 113s 561ms/step - loss: 0.8123 - accuracy: 0.7323 - val_loss: 0.7414 - val_accuracy: 0.7542
Epoch 3/5
201/201 [==============================] - 123s 610ms/step - loss: 0.6362 - accuracy: 0.7747 - val_loss: 0.8563 - val_accuracy: 0.7442
Epoch 4/5
201/201 [==============================] - 112s 556ms/step - loss: 0.5774 - accuracy: 0.7950 - val_loss: 0.7076 - val_accuracy: 0.7598
Epoch 5/5
201/201 [==============================] - 112s 556ms/step - loss: 0.5083 - accuracy: 0.8248 - val_loss: 0.9149 - val_accuracy: 0.7367


In [19]:
import numpy as np

x_test=np.array(x_test).reshape(-1,64,64,3)
score=model.evaluate(x_test,y_test)
print("loss: ",score[0])
print("Accuracy: ",score[1])

63/63 [==============================] - 5s 79ms/step - loss: 0.9060 - accuracy: 0.7389
loss:  0.905958354473114
Accuracy:  0.7388916611671448


In [20]:
classes={0:('akiec', 'actinic keratoses and intraepithelial carcinomae'),
         1:('bcc' , 'basal cell carcinoma'),
         2:('bkl', 'benign keratosis-like lesions'),
         3:('df', 'dermatofibroma'),
         4:('nv', ' melanocytic nevi'),
         5:('vasc', ' pyogenic granulomas and hemorrhage'),
         6:('mel', 'melanoma'),}

In [24]:
import PIL
image=PIL.Image.open('test/aki1.jpg')
image=image.resize((64,64))
img=x_test[1]
img=np.array(image).reshape(-1,64,64,3)
result=model.predict(img)
print(result[0])
result=result.tolist()
max_prob=max(result[0])
class_ind=result[0].index(max_prob)
print(classes[class_ind])

1/1 [==============================] - 0s 18ms/step
[9.5990252e-01 9.3629408e-01 8.4781194e-01 9.3172342e-01 1.2490047e-08
 4.2006694e-02 4.4887567e-01]
('akiec', 'actinic keratoses and intraepithelial carcinomae')


In [26]:
model.save("MobileNet.h5")

In [29]:
from keras.models import load_model
import tensorflow as tf

new_cnn = load_model("MobileNet.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(new_cnn)
tflite_model = converter.convert()

with open("mobilenet.tflite",'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpw792f8rg\assets


INFO:tensorflow:Assets written to: C:\Users\DEEPKH~1\AppData\Local\Temp\tmpw792f8rg\assets


In [28]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)